## [Bayesian Factorization Machines with Stan and R](https://nyhackr.blob.core.windows.net/presentations/A_Common_Model_Separated_by_Two_Disciplines-Adam_Lauretig.pdf)

In [ ]:
library(MASS)
library(rstan)
library(Matrix)
library(trialr)

In [ ]:
options(mc.cores = parallel::detectCores())
rstan_options(auto_write = TRUE)

### Generating Simulation Data

In [ ]:
set.seed(111)

In [ ]:
N <- 100
group_1 <- paste0('i', 1:N)

In [ ]:
group_1

In [ ]:
J <- 20
group_2 <- paste0('j', 1:J)

In [ ]:
group_2

In [ ]:
K <- 5

In [ ]:
predictors <- expand.grid(group_1=group_1, group_2=group_2)

In [ ]:
X_mat <- sparse.model.matrix(~ factor(group_1) + factor(group_2) -1, data=predictors)

In [ ]:
predictors_as_numeric <- cbind(as.numeric(factor(predictors[,1])), as.numeric(factor(predictors[,2])))

In [ ]:
betas <- matrix(rnorm(n=ncol(X_mat), 0,2))

In [ ]:
linear_predictors = X_mat %*% betas  # y

### [LKJ](https://yingqijing.medium.com/lkj-correlation-distribution-in-stan-29927b69e9be)

#### if eta > 1, the correlation values in correlation matrices are going to centered around 0. higher eta indicate no correlations (converge to identity correlation matrix).

![](https://miro.medium.com/max/1400/1*v2LSgSMjYwCjWYpXtk437g.webp)

In [ ]:
gamma_omega <- rlkjcorr(n = 1, K = K, eta = 0.9)#gamma_omega_prior)

In [ ]:
delta_omega <- rlkjcorr(n=1, K=K, eta=0.5)

In [ ]:
gammas <- mvrnorm(n=N, mu=rep(0,K), Sigma = gamma_omega)

In [34]:
deltas <- mvrnorm(n=J, mu=rep(0,K), Sigma = delta_omega)